In [1]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo')

/tmp/ipykernel_103/221477837.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = 'gpt-3.5-turbo')


In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader

!pip install pypdf

!pip install "unstructured[pdf]"

In [4]:
#loader = DirectoryLoader('Data/', glob='**/*.pdf')
#EU_data = loader.load()
#print(len(EU_data))

from langchain.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("CELEX_32016R0679_EN_TXT.pdf")
pdf_pages = pdf_loader.load_and_split()
print(pdf_pages[0])

page_content='I 
(Legislativ e acts) 
REGUL A TIONS 
REGUL A TION (EU) 2016/679 OF THE EUR OPEAN P ARLIAMENT AND OF THE COUNCIL 
of 27 Apr il 2016 
on the protection of natural persons with regard to the processing of personal data and on the free 
mo v ement of such dat a, and repealing Directiv e 95/46/EC (General Data Protection Regulation) 
(T ext with EEA relevance) 
THE EUR OPEAN P ARLIAMENT AND THE COUNCIL OF THE EUR OPEAN UNION, 
Ha ving regar d to the T reaty on the Functioning of the European Union, and in par ticular Ar ticle 16 thereof, 
Ha ving regar d to the proposal from the European Commission, 
Af ter transmission of the draf t legislative act to the national parliaments, 
Ha ving regar d to the opinion of the European Economic and Social Committ ee (
1
), 
Ha ving regar d to the opinion of the Committ ee of the Regions (
2
), 
A cting in accordance with the ordinar y legislative procedure (
3
), 
Whereas: 
(1)  The protection of natural persons in relation to the proc

In [6]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
splits = splitter.split_documents(pdf_pages)
print(len(splits)) 

461


In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [9]:
embedding_model = OpenAIEmbeddings()

/tmp/ipykernel_103/909470085.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


In [10]:
vectorstore = FAISS.from_documents(splits,embedding_model)

In [11]:
vectorstore.save_local("saved_docs")

In [12]:
def retrieval_tool(query):
    retrieved_docs = vectorstore.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in retrieved_docs])

In [13]:
retrieve = retrieval_tool("Who is the president of EU?")
print(retrieve)

author ity of each Member State and the European Data Prote ction Super visor or their respective representatives.

This Regulation shall be binding in its entirety and directly applicable in all Member State s. 
Done at Br ussels, 27 Apr il 2016. 
For the Eur opean P arliament 
The Pres ident 
M. SCH ULZ  
For the Council 
The President 
J.A. HENNIS-PL ASSC HAERT   
4.5.2016 L 119/88 Official Jour nal of the European Union EN

Chapt er VII. 
3. Where more than one super visor y author ity is established in a Member Stat e, that Member State shall designat e 
the super visor y author ity which is to represent those author ities in the Board and shall set out the mechanis m to ensure 
compl iance by the other author ities with the r ules relating to the consiste ncy mechani sm refe r red to in Ar ticle 63. 
4. Each Member Stat e shall notify to the Commission the pro visions of its law which it adopts pursuant to this 
Chapt er , by 25 Ma y 2018 and, without dela y , an y subsequent ame

In [14]:
from langchain.agents import initialize_agent,  AgentType
from langchain.tools import Tool

In [15]:
tools = [
    #Tool(name="RephraseTool", func=rephrasing_tool, description="Rephrase the search query"),
    Tool(name="RetrieveTool", func=retrieval_tool, description="Retrieve the searched result from the embeddings"),
    #Tool(name="ReranTool", func=re_ranker_tool, description="Rerank retrieved documents to pick best matches")
]

In [16]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)

/tmp/ipykernel_103/3437836968.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)


In [17]:
agents = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools, 
    llm=llm,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

/tmp/ipykernel_103/3931876160.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agents = initialize_agent(


In [18]:
agents.invoke("Who is the president of EU?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RetrieveTool
Action Input: President of EU
Observation: This Regulation shall be binding in its entirety and directly applicable in all Member State s. 
Done at Br ussels, 27 Apr il 2016. 
For the Eur opean P arliament 
The Pres ident 
M. SCH ULZ  
For the Council 
The President 
J.A. HENNIS-PL ASSC HAERT   
4.5.2016 L 119/88 Official Jour nal of the European Union EN

author ity of each Member State and the European Data Prote ction Super visor or their respective representatives.

av ailable to the public, to the Commission and to the Board. 
4.5.2016 L 119/70 Official Jour nal of the European Union EN
Thought:Could not parse LLM output: `The President of the European Union is not a single person like in some countries, but rather the EU is led by several institutions, including the European Council and the European Commission. The President of the European Council is Charles Michel, and the Preside

{'input': 'Who is the president of EU?',
 'chat_history': [HumanMessage(content='Who is the president of EU?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The President of the European Union is not a single person like in some countries, but rather the EU is led by several institutions, including the European Council and the European Commission. The President of the European Council is Charles Michel, and the President of the European Commission is Ursula von der Leyen.', additional_kwargs={}, response_metadata={})],
 'output': 'The President of the European Union is not a single person like in some countries, but rather the EU is led by several institutions, including the European Council and the European Commission. The President of the European Council is Charles Michel, and the President of the European Commission is Ursula von der Leyen.'}